In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

path='/caldera/projects/usgs/water/impd/onhm/historical/output/seg_outflow.nc'
stream_shape='/caldera/projects/usgs/water/impd/onhm/GIS/streamSegsAllConus/segsAllConus.shp'
hru_shape='/caldera/projects/usgs/water/impd/onhm/GIS/hrusAllConus/hrusAllConus.shp'

In [ ]:
    # path is the path to were there are output PRMS ncf files
    # ext is the file extention on these files
    # The python package "onhm" can be built from the source code at https://github.com/nhm-usgs/pangeo/src/onhm
    
    import onhm
    
    # For example, to see the results of the historical ONHM runs on Denali, the path would be
    path = '/caldera/projects/usgs/water/impd/onhm/historical/output/'
    #path = '/work/markstro/operat/setup/test/NHM-PRMS_CONUS/output/'
    ext = '.nc'
    
    ds_out = onhm.reader.get_DataSet(path, ext)
    print(ds_out)

In [ ]:
    # Get the values for all HRUs for an output variable for a single time step
    var_name = 'seg_outflow'
    time_step = '2000-06-01'
    seg_outflow_sel = ds_out[var_name].sel(time=time_step)
    
    # This is an xarray DataArray
    print(seg_outflow_sel)
    # This is a numpy ndarray
    print("numpy.ndarray=", seg_outflow_sel.values)
    print('number of hrus = ', seg_outflow_sel.values.size)

In [ ]:
streams = gpd.read_file(stream_shape)
streams.head()

In [ ]:
vals = seg_outflow_sel.values

streams['outflow'] = vals[0]
streams.head()

In [ ]:
coolwarm = cm.get_cmap('coolwarm', 12)

In [ ]:
fig, ax = plt.subplots(1, 1)
streams.plot(column='outflow', cmap='coolwarm', ax=ax, legend=True)

In [ ]:
foo = vals[0]
foo[foo < .1] = .1

foo_ln = np.log(foo) 
streams['outflow_ln'] = foo_ln
streams.head()

In [ ]:
fig, ax = plt.subplots(1, 1)
streams.plot(column='outflow_ln', cmap='coolwarm', ax=ax, legend=True)